In [30]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_absolute_error as mean_squared_error
from math import sqrt

In [2]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm

In [3]:
df = pd.read_stata("../data/variables_productos_primarios.dta",index_col="fecha")
df = df.drop(["mes","anio"],axis=1)
df["pp"] = df.pp / 1_000_000
df.index = df.index.to_period('M')
df

,itcr,ip,pre,pp
fecha,,,,
2005-01,105.929359,86.866026,1423.333333,508.020737
2005-02,103.960244,87.907449,969.666667,500.539941
2005-03,102.549715,86.226525,1730.000000,656.181802
2005-04,101.267876,87.964450,620.000000,939.064907
2005-05,100.294069,89.117152,54.666667,1028.823820
...,...,...,...,...
2022-07,44.102487,208.916689,0.000000,2431.418408
2022-08,43.439754,200.965695,146.666667,1988.408046
2022-09,43.289343,205.021010,106.666667,2071.879245


In [33]:
#43 datos para testeo si n_splits = 4, sino 35 (5)
#Predecir usando train como insumo, comparar sus predicciones con los de test (MSE)
#La prediccion tiene que ser los proximos 35 valores. 
#Hacer la media del MSE
X = ["itcr","ip","pre"]
tscv = TimeSeriesSplit(n_splits = 5)
RMSE = []
for train_index, test_index in tscv.split(df):
    cv_train, cv_test = df.iloc[train_index],df.iloc[test_index]
    # print(test_index.shape, "...\n")
    meses_prediccion = cv_test.shape[0]
    sarima_exog = SARIMAX(cv_train["pp"], order = (0,0,0),exog=cv_train[X], seasonal_order=(0,0,0,12))
    model_fit = sarima_exog.fit(maxiter=20_000)
    predictions = model_fit.forecast(meses_prediccion, exog = cv_test[X])
    rmse_split = sqrt(mean_squared_error(cv_test["pp"], predictions))
    RMSE.append(rmse_split)
    
np.mean(RMSE)
    
    
    

19.082307668791987